In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

import os

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [3]:
mlflow.get_tracking_uri()

'127.0.0.1'

In [4]:
sentence_length = 106

run_experiments = os.getenv("UVL_BERT_RUN_EXPERIMENTS", "True") == "True"
pin_commits = os.getenv("UVL_BERT_PIN_COMMITS", "True") == "FALSE"

print(f"{run_experiments=}")
print(f"{pin_commits=}")

run_experiments=True
pin_commits=False


In [5]:
from tooling.config import Experiment, Transformation
#Set Iterations=1 for Split training and testing on datasets. Set dataset to desired dataset split. See tooling.py for split options
base_experiment_config = Experiment(
    name="Base Config 1",
    iterations=1,
    force=False,
    dataset="all",
    lower_case=True,
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Domain_Level",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Domain_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

In [6]:
from tooling.transformation import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)

Path("./src/service/models/").mkdir(parents=True, exist_ok=True)

hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

12:29:47 INFO:Hint Label2Id: hint_label2id={'0': 0, 'System_Level': 1, 'Domain_Level': 2, 'Interaction_Level': 3}
12:29:47 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Task': 1, 'Domain_Data': 2, 'Activity': 3, 'Stakeholder': 4, 'System_Function': 5, 'Interaction': 6, 'Interaction_Data': 7, 'Workspace': 8, 'System_Level': 9}


## Train BiLSTM First Stage Model


In [9]:
'''
from tooling.observability import get_run_id
from tooling.config import BiLSTMConfig, BiLSTM

from copy import deepcopy

bilstm_experiment_config = deepcopy(base_experiment_config)
bilstm_experiment_config.name = "Production"

bilstm_config = BiLSTM(
    batch_size=16,
    learning_rate=0.006,
    number_epochs=5,
    weighted_classes=False,
    sentence_length=106,
)

bilstm_cfg: BiLSTMConfig = OmegaConf.structured(
    BiLSTMConfig(
        bilstm=bilstm_config,
        experiment=bilstm_experiment_config,
        transformation=levels_transformation_config,
    )
)

# bilstm_cfg.experiment.force = True

if run_experiments:
    from experiments.bilstm import bilstm

    bilstm(bilstm_cfg)


bilstm_run_id = get_run_id(bilstm_cfg, pin_commit=pin_commits)

print(bilstm_run_id)
print(mlflow.get_artifact_uri())

bilstm_run = mlflow.get_run(bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bilstm")
'''

01:03:24 INFO:
bilstm:
  type: BiLSTM
  sentence_length: 106
  batch_size: 16
  number_epochs: 5
  verbose: 1
  weighted_classes: false
  learning_rate: 0.006
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
  smote_balance_to_average: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Domain_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

01:03:24 WARNING:Experiment was already run


ec764d893a574ea0a71bcdd67e025dc5
file:///E:/BERT/uvl-tore-classifier-bert/127.0.0.1/279074982670971635/7fdf5594883148b6ada0cb314bd89b86/artifacts


WindowsPath('src/service/models/bilstm')

## Train SNER First Stage Model


In [13]:
'''
from tooling.observability import (get_run_id)
from tooling.config import SNERConfig, SNER

from copy import deepcopy

sner_experiment_config = deepcopy(base_experiment_config)
sner_experiment_config.name = "Production"

sner_config = SNER()

sner_cfg = OmegaConf.structured(
    SNERConfig(
        sner=sner_config,
        experiment=sner_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.sner import sner

    sner(OmegaConf.create(sner_cfg))

sner_run_id = get_run_id(sner_cfg, pin_commit=pin_commits)
print(mlflow.get_artifact_uri())
print(sner_run_id)

sner_run = mlflow.get_run(sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{sner_run.info.artifact_uri}/0_model.ser.gz",
    dst_path=Path("./src/service/models/"),
)
try:
    Path("./src/service/models/sner.ser.gz").unlink()
except FileNotFoundError:
    pass
Path("./src/service/models/0_model.ser.gz").rename(
    "./src/service/models/sner.ser.gz"
)
'''

SyntaxError: unterminated triple-quoted string literal (detected at line 40) (2848408053.py, line 1)

## Train BERT First Stage Model


In [7]:
'''
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_1_experiment_config = deepcopy(base_experiment_config)
bert_1_experiment_config.name = "Production"
bert_1_experiment_config.force = False

bert_1_config = BERT(
    max_len=123,
    number_epochs=11,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=True,
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.0005,
    validation_batch_size=64,
)


bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg, pin_commit=pin_commits)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")
## Train BERT Second Stage Model for BERT First Stage
'''

12:30:11 INFO:PyTorch version 2.3.0+cu118 available.
12:30:11 INFO:TensorFlow version 2.15.1 available.
12:30:39 WARNING:From E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

12:30:40 INFO:
bert:
  model: bert-large-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 64
  number_epochs: 11
  learning_rate_bert: 3.0e-05
  learning_rate_classifier: 0.0005
  weight_decay: 0.01
  weighted_classes: true
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
  smote_balance_to_average: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: 

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
12:32:05 INFO:Logged iteration result res.precision=0.6756897999233451 res.recall=0.8142645295054445


{'eval_loss': 0.5253523588180542, 'eval_step': 0, 'eval_precision': 0.6756897999233451, 'eval_recall': 0.8142645295054445, 'eval_f1': 0.7385330223338313, 'eval_label_count': 4, 'eval_runtime': 3.7893, 'eval_samples_per_second': 164.148, 'eval_steps_per_second': 2.639, 'epoch': 1.0}
{'loss': 0.5786, 'grad_norm': 4.1576666831970215, 'learning_rate': 2.561531715872552e-05, 'epoch': 1.607717041800643}


...

12:49:08 INFO:Logged iteration result res.precision=0.8057886955001589 res.recall=0.8106319383455668


{'eval_loss': 1.6497583389282227, 'eval_step': 20, 'eval_precision': 0.8057886955001589, 'eval_recall': 0.8106319383455668, 'eval_f1': 0.8082030610759724, 'eval_label_count': 4, 'eval_runtime': 3.4679, 'eval_samples_per_second': 179.359, 'eval_steps_per_second': 2.884, 'epoch': 11.0}
{'train_runtime': 1131.9258, 'train_samples_per_second': 24.178, 'train_steps_per_second': 3.022, 'train_loss': 0.15274266860736668, 'epoch': 11.0}


12:50:10 INFO:Logged iteration result res.precision=0.8057886955001589 res.recall=0.8106319383455668
12:50:10 INFO:Logged iteration result res.precision=0.8057886955001589 res.recall=0.8106319383455668
12:50:10 INFO:Finished iteration=0
12:50:10 INFO:Logging model artifact (might take a while)
12:50:14 INFO:Breaking early after iteration=0 of 5 folds
12:50:15 INFO:Logged experiment result res.mean_precision=0.8057886955001589 res.mean_recall=0.8106319383455668
12:50:16 INFO:Left mlflow context


d3fc119779a74a40b6010a0de4a8ab79


WindowsPath('src/service/models/bert_1')

In [8]:
'''
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bert_experiment_config = deepcopy(base_experiment_config)
bert_2_bert_experiment_config.name = "Production"

bert_2_bert_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=8,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
    validation_batch_size=64,
)

cfg = DualModelStagedBERTConfig(
    bert=bert_2_bert_config,
    experiment=bert_2_bert_experiment_config,
    transformation=label_transformation_config,
    first_model_bert=bert_1_cfg,
)

bert_2_bert_cfg: DualModelStagedBERTConfig = OmegaConf.structured(cfg)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_bert_cfg))

bert_2_bert_run_id = get_run_id(bert_2_bert_cfg, pin_commit=pin_commits)

print(bert_2_bert_run_id)

bert_2_bert_run = mlflow.get_run(bert_2_bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bert_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_2_bert")
'''

12:55:01 INFO:
first_model_bert:
  bert:
    model: bert-large-uncased
    type: BERT
    max_len: 123
    train_batch_size: 8
    validation_batch_size: 64
    number_epochs: 11
    learning_rate_bert: 3.0e-05
    learning_rate_classifier: 0.0005
    weight_decay: 0.01
    weighted_classes: true
  experiment:
    name: Production
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: all
    lower_case: true
    force: false
    pin_commit: false
    smote: false
    smote_k_neighbors: 5
    smote_sampling_strategy: not majority
    smote_balance_to_average: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Domain_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: Sy

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

12:55:08 INFO:Loading Model
12:55:10 INFO:Creating hint column
12:55:25 INFO:Hint column created


Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

...

01:01:09 INFO:Logged iteration result res.precision=0.6177489587915692 res.recall=0.6419935353316357


{'eval_loss': 3.0090813636779785, 'eval_step': 14, 'eval_precision': 0.6177489587915692, 'eval_recall': 0.6419935353316357, 'eval_f1': 0.629637945615335, 'eval_label_count': 10, 'eval_runtime': 1.6695, 'eval_samples_per_second': 372.577, 'eval_steps_per_second': 5.99, 'epoch': 8.0}
{'train_runtime': 352.3917, 'train_samples_per_second': 56.483, 'train_steps_per_second': 7.06, 'train_loss': 2.1583940070541727, 'epoch': 8.0}


01:01:33 INFO:Logged iteration result res.precision=0.6177489587915692 res.recall=0.6419935353316357
01:01:34 INFO:Logged iteration result res.precision=0.6177489587915692 res.recall=0.6419935353316357
01:01:34 INFO:Finished iteration=0
01:01:34 INFO:Logging model artifact (might take a while)
01:01:36 INFO:Breaking early after iteration=0 of 5 folds
01:01:37 INFO:Logged experiment result res.mean_precision=0.6177489587915692 res.mean_recall=0.6419935353316357
01:01:38 INFO:Left mlflow context


ef1de5d973ba48da908754e23427c330


WindowsPath('src/service/models/bert_2_bert')

## Train BERT Second Stage Model for SNER First Stage


In [ ]:
'''
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"

bert_2_sner_experiment_config = deepcopy(base_experiment_config)
bert_2_sner_experiment_config.name = "Production"

bert_2_sner_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=5,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
)

bert_2_sner_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_sner_config,
        experiment=bert_2_sner_experiment_config,
        transformation=label_transformation_config,
        first_model_sner=sner_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_sner_cfg))

bert_2_sner_run_id = get_run_id(bert_2_sner_cfg, pin_commit=pin_commits)

print(bert_2_sner_run_id)

bert_2_sner_run = mlflow.get_run(bert_2_sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_sner_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_sner"))
except FileNotFoundError:
    pass

Path("./src/service/models/0_model").rename("./src/service/models/bert_2_sner")
'''

## Train BERT Second Stage Model for BILSTM First Stage


In [10]:
'''
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bilstm_config = BiLSTM(
    batch_size=16,
    learning_rate=0.006,
    number_epochs=5,
    weighted_classes=False,
    sentence_length=106,
)


bert_2_bilstm_experiment_config = deepcopy(base_experiment_config)
bert_2_bilstm_experiment_config.name = "Production"

bert_2_bilstm_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=5,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
)

bert_2_bilstm_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bilstm_config,
        experiment=bert_2_bilstm_experiment_config,
        transformation=label_transformation_config,
        first_model_bilstm=bilstm_cfg,
    )
)
bert_2_bilstm_cfg.first_model_bilstm.bilstm.sentence_length = 106


if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_bilstm_cfg))

bert_2_bilstm_run_id = get_run_id(bert_2_bilstm_cfg, pin_commit=pin_commits)

print(bert_2_bilstm_run_id)

bert_2_bilstm_run = mlflow.get_run(bert_2_bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename(
    "./src/service/models/bert_2_bilstm"
)
'''

01:04:18 INFO:
first_model_bert: null
first_model_bilstm:
  bilstm:
    type: BiLSTM
    sentence_length: 106
    batch_size: 16
    number_epochs: 5
    verbose: 1
    weighted_classes: false
    learning_rate: 0.006
  experiment:
    name: Production
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: all
    lower_case: true
    force: false
    pin_commit: false
    smote: false
    smote_k_neighbors: 5
    smote_sampling_strategy: not majority
    smote_balance_to_average: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Domain_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
    system_level: System_Level
first_

01:04:21 INFO:loading projection weights from C:\Users\mjand/gensim-data\glove-twitter-100\glove-twitter-100.gz
01:05:07 INFO:KeyedVectors lifecycle event {'msg': 'loaded (1193514, 100) matrix of type float32 from C:\\Users\\mjand/gensim-data\\glove-twitter-100\\glove-twitter-100.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2024-08-09T13:05:07.502736', 'gensim': '4.3.2', 'python': '3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'load_word2vec_format'}
01:05:08 WARNING:From E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\keras\src\saving\legacy\saved_model\load.py:107: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

01:05:09 WARNING:From E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\keras\src\engine\functional.py:156: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_function

78/78 [==============================] - 4s 14ms/step


Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

20/20 [==============================] - 1s 16ms/step


...

01:09:23 INFO:Logged iteration result res.precision=0.4494197880174009 res.recall=0.6317989450945161


{'eval_loss': 1.3167644739151, 'eval_step': 8, 'eval_precision': 0.4494197880174009, 'eval_recall': 0.6317989450945161, 'eval_f1': 0.5252275775073979, 'eval_label_count': 10, 'eval_runtime': 1.7127, 'eval_samples_per_second': 363.165, 'eval_steps_per_second': 11.677, 'epoch': 5.0}
{'train_runtime': 200.609, 'train_samples_per_second': 62.011, 'train_steps_per_second': 7.751, 'train_loss': 3.755339663588349, 'epoch': 5.0}


01:09:45 INFO:Logged iteration result res.precision=0.4494197880174009 res.recall=0.6317989450945161
01:09:45 INFO:Logged iteration result res.precision=0.4494197880174009 res.recall=0.6317989450945161
01:09:45 INFO:Finished iteration=0
01:09:45 INFO:Logging model artifact (might take a while)
01:09:46 INFO:Breaking early after iteration=0 of 5 folds
01:09:47 INFO:Logged experiment result res.mean_precision=0.4494197880174009 res.mean_recall=0.6317989450945161
01:09:48 INFO:Left mlflow context


f21fd79e52d647b899146124dadb3f3d


WindowsPath('src/service/models/bert_2_bilstm')

## Train BERT E2E Model


In [11]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_experiment_config = deepcopy(base_experiment_config)
bert_experiment_config.name = "Production"

bert_config = BERT(
    learning_rate_bert=6e-05,
    learning_rate_classifier=0.1,
    max_len=123,
    number_epochs=11 ,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg, pin_commit=pin_commits)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")

01:42:10 INFO:
bert:
  model: bert-large-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 32
  number_epochs: 11
  learning_rate_bert: 6.0e-05
  learning_rate_classifier: 0.1
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
  smote_balance_to_average: false
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

01:42:11 INFO:New experiment. Running
01:42:11 INFO:Entering

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
01:43:23 INFO:Logged iteration result res.precision=0.6213213662604519 res.recall=0.5905146054557319


{'eval_loss': 0.5869194269180298, 'eval_step': 0, 'eval_precision': 0.6213213662604519, 'eval_recall': 0.5905146054557319, 'eval_f1': 0.6055264078997584, 'eval_label_count': 10, 'eval_runtime': 3.8157, 'eval_samples_per_second': 163.011, 'eval_steps_per_second': 5.242, 'epoch': 1.0}
{'loss': 0.8892, 'grad_norm': 17.129610061645508, 'learning_rate': 5.123063431745104e-05, 'epoch': 1.607717041800643}


...

02:00:34 INFO:Logged iteration result res.precision=0.6679785896915205 res.recall=0.6638011770091515


{'eval_loss': 1.2225558757781982, 'eval_step': 20, 'eval_precision': 0.6679785896915205, 'eval_recall': 0.6638011770091515, 'eval_f1': 0.6658833316752186, 'eval_label_count': 10, 'eval_runtime': 3.9822, 'eval_samples_per_second': 156.194, 'eval_steps_per_second': 5.022, 'epoch': 11.0}
{'train_runtime': 1138.3761, 'train_samples_per_second': 24.041, 'train_steps_per_second': 3.005, 'train_loss': 0.24518448035990903, 'epoch': 11.0}


02:01:24 INFO:Logged iteration result res.precision=0.6679785896915205 res.recall=0.6638011770091515
02:01:24 INFO:Logged iteration result res.precision=0.6679785896915205 res.recall=0.6638011770091515
02:01:24 INFO:Finished iteration=0
02:01:24 INFO:Logging model artifact (might take a while)
02:01:26 INFO:Breaking early after iteration=0 of 5 folds
02:01:27 INFO:Logged experiment result res.mean_precision=0.6679785896915205 res.mean_recall=0.6638011770091515
02:01:28 INFO:Left mlflow context


be26e295cb9d4b5c9dbbbe6fb2396c22


WindowsPath('src/service/models/bert')

In [ ]:
##Train RoBERTa E2E Model


In [14]:
'''
from tooling.observability import get_run_id
from tooling.config import RoBERTaConfig, RoBERTa

from copy import deepcopy

roberta_experiment_config = deepcopy(base_experiment_config)
roberta_experiment_config.name = "Production"

roberta_config = RoBERTa(
    learning_rate_roberta=6e-05,
    learning_rate_classifier=0.1,
    max_len=123,
    number_epochs=11,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
)

roberta_cfg = OmegaConf.structured(
    RoBERTaConfig(
        roberta=roberta_config,
        experiment=roberta_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.roberta import roberta

    roberta(OmegaConf.create(roberta_cfg))

roberta_run_id = get_run_id(roberta_cfg, pin_commit=pin_commits)

print(roberta_run_id)

run = mlflow.get_run(roberta_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/roberta"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/roberta")
'''

01:44:16 INFO:
roberta:
  model: Jean-Baptiste/roberta-large-ner-english
  type: RoBERTa
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 32
  number_epochs: 11
  learning_rate_roberta: 6.0e-05
  learning_rate_classifier: 0.1
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

01:44:16 INFO:New experiment. Running
01:44:16 INFO:Entering mlf

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
01:45:21 INFO:Logged iteration result res.precision=0.5793384603014975 res.recall=0.5294931132606071


{'eval_loss': 0.606141209602356, 'eval_step': 0, 'eval_precision': 0.5793384603014975, 'eval_recall': 0.5294931132606071, 'eval_f1': 0.5532954369096804, 'eval_label_count': 10, 'eval_runtime': 3.4457, 'eval_samples_per_second': 180.515, 'eval_steps_per_second': 5.804, 'epoch': 1.0}
{'loss': 1.2792, 'grad_norm': 15.624954223632812, 'learning_rate': 5.123063431745104e-05, 'epoch': 1.607717041800643}


...

02:01:44 INFO:Logged iteration result res.precision=0.6904512276980231 res.recall=0.6926464362913085


{'eval_loss': 0.9886234998703003, 'eval_step': 20, 'eval_precision': 0.6904512276980231, 'eval_recall': 0.6926464362913085, 'eval_f1': 0.6915470899120589, 'eval_label_count': 10, 'eval_runtime': 3.5215, 'eval_samples_per_second': 176.631, 'eval_steps_per_second': 5.679, 'epoch': 11.0}
{'train_runtime': 1083.2292, 'train_samples_per_second': 25.265, 'train_steps_per_second': 3.158, 'train_loss': 0.45614679554546483, 'epoch': 11.0}


02:02:35 INFO:Logged iteration result res.precision=0.6904512276980231 res.recall=0.6926464362913085
02:02:35 INFO:Logged iteration result res.precision=0.6904512276980231 res.recall=0.6926464362913085
02:02:35 INFO:Finished iteration=0
02:02:35 INFO:Logging model artifact (might take a while)
02:02:38 INFO:Breaking early after iteration=0 of 5 folds
02:02:38 INFO:Logged experiment result res.mean_precision=0.6904512276980231 res.mean_recall=0.6926464362913085
02:02:39 INFO:Left mlflow context


73c3721d2c3e41e6b0539148725132df


WindowsPath('src/service/models/roberta')